# Data Import

## Introduction

This notebook shows how to import data into a DuckDB database. It uses the `duckdb` Python package to connect to a DuckDB database and import data from various formats, including CSV, JSON, DataFrame, parquet, GeoJSON, Shapefile, GeoParquet, and more.

## Datasets

The following datasets are used in this notebook. You don't need to download them, they can be accessed directly from the notebook.

- [cities.csv](https://open.gishub.org/data/duckdb/cities.csv)
- [countries.csv](https://open.gishub.org/data/duckdb/countries.csv)

## Installation

Uncomment the following cell to install the required packages if needed.

In [1]:
%pip install duckdb leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.5/765.5 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.9 MB/s eta 0:00:00


## Library Import

In [2]:
import duckdb
import leafmap
import pandas as pd

## Installing Extensions

DuckDB’s Python API provides functions for installing and loading extensions, which perform the equivalent operations to running the `INSTALL` and `LOAD` SQL commands, respectively. An example that installs and loads the [httpfs extension](https://duckdb.org/docs/extensions/httpfs) looks like follows:

In [3]:
con = duckdb.connect()

In [4]:
con.install_extension("httpfs")
con.load_extension("httpfs")

In [5]:
con.install_extension("spatial")
con.load_extension("spatial")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Downloading Sample Data

In [6]:
url = "https://open.gishub.org/data/duckdb/cities.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://open.gishub.org/data/duckdb/cities.zip
To: /content/cities.zip
100%|██████████| 214k/214k [00:00<00:00, 1.35MB/s]

Extracting files...


'/content/cities.zip'

## CSV Files

CSV files can be read using the `read_csv` function, called either from within Python or directly from within SQL. By default, the `read_csv` function attempts to auto-detect the CSV settings by sampling from the provided file.

In [7]:
# read from a file using fully auto-detected settings
con.read_csv('cities.csv')

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │
│     6 │ Mariehamn        │ ALD     │    60.097 │    19.949 │      10682 │
│     7 │ Ramallah         │ PSE     │  31.90294 │  35.20621 │      24599 │
│     8 │ Vatican City     │ VAT     │  41.90001 │  12.44781 │        832 │
│     9 │ Poitier          │ FRA     │  46.58329 │   0.33328 │      85960 │
│    10 │ Cl

In [8]:
# specify options on how the CSV is formatted internally
con.read_csv('cities.csv', header=True, sep=',')

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │
│     6 │ Mariehamn        │ ALD     │    60.097 │    19.949 │      10682 │
│     7 │ Ramallah         │ PSE     │  31.90294 │  35.20621 │      24599 │
│     8 │ Vatican City     │ VAT     │  41.90001 │  12.44781 │        832 │
│     9 │ Poitier          │ FRA     │  46.58329 │   0.33328 │      85960 │
│    10 │ Cl

In [9]:
# use the (experimental) parallel CSV reader
con.read_csv('cities.csv', parallel=True)

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │
│     6 │ Mariehamn        │ ALD     │    60.097 │    19.949 │      10682 │
│     7 │ Ramallah         │ PSE     │  31.90294 │  35.20621 │      24599 │
│     8 │ Vatican City     │ VAT     │  41.90001 │  12.44781 │        832 │
│     9 │ Poitier          │ FRA     │  46.58329 │   0.33328 │      85960 │
│    10 │ Cl

In [10]:
# directly read a CSV file from within SQL
con.sql("SELECT * FROM 'cities.csv'")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │
│     6 │ Mariehamn        │ ALD     │    60.097 │    19.949 │      10682 │
│     7 │ Ramallah         │ PSE     │  31.90294 │  35.20621 │      24599 │
│     8 │ Vatican City     │ VAT     │  41.90001 │  12.44781 │        832 │
│     9 │ Poitier          │ FRA     │  46.58329 │   0.33328 │      85960 │
│    10 │ Cl

In [11]:
# call read_csv from within SQL
con.sql("SELECT * FROM read_csv_auto('cities.csv')")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │
│     6 │ Mariehamn        │ ALD     │    60.097 │    19.949 │      10682 │
│     7 │ Ramallah         │ PSE     │  31.90294 │  35.20621 │      24599 │
│     8 │ Vatican City     │ VAT     │  41.90001 │  12.44781 │        832 │
│     9 │ Poitier          │ FRA     │  46.58329 │   0.33328 │      85960 │
│    10 │ Cl

## JSON Files

JSON files can be read using the `read_json` function, called either from within Python or directly from within SQL. By default, the `read_json` function will automatically detect if a file contains newline-delimited JSON or regular JSON, and will detect the schema of the objects stored within the JSON file.

In [12]:
# read from a single JSON file
con.read_json('cities.json')

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │
│     6 │ Mariehamn        │ ALD     │    60.097 │    19.949 │      10682 │
│     7 │ Ramallah         │ PSE     │  31.90294 │  35.20621 │      24599 │
│     8 │ Vatican City     │ VAT     │  41.90001 │  12.44781 │        832 │
│     9 │ Poitier          │ FRA     │  46.58329 │   0.33328 │      85960 │
│    10 │ Cl

In [13]:
# directly read a JSON file from within SQL
con.sql("SELECT * FROM 'cities.json'")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │
│     6 │ Mariehamn        │ ALD     │    60.097 │    19.949 │      10682 │
│     7 │ Ramallah         │ PSE     │  31.90294 │  35.20621 │      24599 │
│     8 │ Vatican City     │ VAT     │  41.90001 │  12.44781 │        832 │
│     9 │ Poitier          │ FRA     │  46.58329 │   0.33328 │      85960 │
│    10 │ Cl

In [14]:
# call read_json from within SQL
con.sql("SELECT * FROM read_json_auto('cities.json')")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │
│     6 │ Mariehamn        │ ALD     │    60.097 │    19.949 │      10682 │
│     7 │ Ramallah         │ PSE     │  31.90294 │  35.20621 │      24599 │
│     8 │ Vatican City     │ VAT     │  41.90001 │  12.44781 │        832 │
│     9 │ Poitier          │ FRA     │  46.58329 │   0.33328 │      85960 │
│    10 │ Cl

## DataFrames

DuckDB is automatically able to query a Pandas DataFrame.

In [15]:
df = pd.read_csv('cities.csv')
df

,id,name,country,latitude,longitude,population
0,1,Bombo,UGA,0.58330,32.53330,75000
1,2,Fort Portal,UGA,0.67100,30.27500,42670
2,3,Potenza,ITA,40.64200,15.79900,69060
3,4,Campobasso,ITA,41.56300,14.65600,50762
4,5,Aosta,ITA,45.73700,7.31500,34062
...,...,...,...,...,...,...
1244,1245,Rio de Janeiro,BRA,-22.92502,-43.22502,11748000
1245,1246,Sao Paulo,BRA,-23.55868,-46.62502,18845000
1246,1247,Sydney,AUS,-33.92001,151.18518,4630000
1247,1248,Singapore,SGP,1.29303,103.85582,5183700


from matplotlib import pyplot as plt
df['id'].plot(kind='hist', bins=20, title='id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['latitude'].plot(kind='hist', bins=20, title='latitude')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['longitude'].plot(kind='hist', bins=20, title='longitude')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['population'].plot(kind='hist', bins=20, title='population')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['id']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'id'}, axis=1)
              .sort_values('id', ascending=True))
  xs = counted['id']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df.sort_values('id', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('id')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
df['id'].plot(kind='line', figsize=(8, 4), title='id')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['latitude'].plot(kind='line', figsize=(8, 4), title='latitude')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['longitude'].plot(kind='line', figsize=(8, 4), title='longitude')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['population'].plot(kind='line', figsize=(8, 4), title='population')
plt.gca().spines[['top', 'right']].set_visible(False)

In [16]:
con.sql('SELECT * FROM df').fetchall()

[(1, 'Bombo', 'UGA', 0.5833, 32.5333, 75000),
 (2, 'Fort Portal', 'UGA', 0.671, 30.275, 42670),
 (3, 'Potenza', 'ITA', 40.642, 15.799, 69060),
 (4, 'Campobasso', 'ITA', 41.563, 14.656, 50762),
 (5, 'Aosta', 'ITA', 45.737, 7.315, 34062),
 (6, 'Mariehamn', 'ALD', 60.097, 19.949, 10682),
 (7, 'Ramallah', 'PSE', 31.90294, 35.20621, 24599),
 (8, 'Vatican City', 'VAT', 41.90001, 12.44781, 832),
 (9, 'Poitier', 'FRA', 46.58329, 0.33328, 85960),
 (10, 'Clermont-Ferrand', 'FRA', 45.77998, 3.08001, 233050),
 (11, 'Besancon', 'FRA', 47.23, 6.03001, 128426),
 (12, 'Artigas Base', 'ATA', -62.17389, -58.86386, 60),
 (13, 'Chipata', 'ZMB', -13.62957, 32.64001, 85963),
 (14, 'Jinja', 'UGA', 0.44042, 33.19993, 301619),
 (15, 'Arua', 'UGA', 3.02037, 30.90002, 250000),
 (16, 'Mbale', 'UGA', 1.09041, 34.17, 402368),
 (17, 'Moroto', 'UGA', 2.54035, 34.63999, 371),
 (18, 'Masaka', 'UGA', -0.32961, 31.72999, 65373),
 (19, 'Mbarara', 'UGA', -0.59962, 30.65, 83700),
 (20, 'Otjiwarongo', 'NAM', -20.45954, 16.64

## Parquet Files

Parquet files can be read using the `read_parquet` function, called either from within Python or directly from within SQL.

In [19]:
# read from a single Parquet file
con.read_parquet('cities.parquet')

┌─────────┬─────────────────────────────┬────────┬───────────┬───────────┬──────────────────┬────────────┐
│ country │          geometry           │   id   │ latitude  │ longitude │       name       │ population │
│ varchar │          geometry           │ double │  double   │  double   │     varchar      │   double   │
├─────────┼─────────────────────────────┼────────┼───────────┼───────────┼──────────────────┼────────────┤
│ UGA     │ POINT (32.5333 0.5833)      │    1.0 │    0.5833 │   32.5333 │ Bombo            │    75000.0 │
│ UGA     │ POINT (30.275 0.671)        │    2.0 │     0.671 │    30.275 │ Fort Portal      │    42670.0 │
│ ITA     │ POINT (15.799 40.642)       │    3.0 │    40.642 │    15.799 │ Potenza          │    69060.0 │
│ ITA     │ POINT (14.656 41.563)       │    4.0 │    41.563 │    14.656 │ Campobasso       │    50762.0 │
│ ITA     │ POINT (7.315 45.737)        │    5.0 │    45.737 │     7.315 │ Aosta            │    34062.0 │
│ ALD     │ POINT (19.949 60.097)    

In [20]:
# directly read a Parquet file from within SQL
con.sql("SELECT * FROM 'cities.parquet'")

┌─────────┬─────────────────────────────┬────────┬───────────┬───────────┬──────────────────┬────────────┐
│ country │          geometry           │   id   │ latitude  │ longitude │       name       │ population │
│ varchar │          geometry           │ double │  double   │  double   │     varchar      │   double   │
├─────────┼─────────────────────────────┼────────┼───────────┼───────────┼──────────────────┼────────────┤
│ UGA     │ POINT (32.5333 0.5833)      │    1.0 │    0.5833 │   32.5333 │ Bombo            │    75000.0 │
│ UGA     │ POINT (30.275 0.671)        │    2.0 │     0.671 │    30.275 │ Fort Portal      │    42670.0 │
│ ITA     │ POINT (15.799 40.642)       │    3.0 │    40.642 │    15.799 │ Potenza          │    69060.0 │
│ ITA     │ POINT (14.656 41.563)       │    4.0 │    41.563 │    14.656 │ Campobasso       │    50762.0 │
│ ITA     │ POINT (7.315 45.737)        │    5.0 │    45.737 │     7.315 │ Aosta            │    34062.0 │
│ ALD     │ POINT (19.949 60.097)    

In [21]:
# call read_parquet from within SQL
con.sql("SELECT * FROM read_parquet('cities.parquet')")

┌─────────┬─────────────────────────────┬────────┬───────────┬───────────┬──────────────────┬────────────┐
│ country │          geometry           │   id   │ latitude  │ longitude │       name       │ population │
│ varchar │          geometry           │ double │  double   │  double   │     varchar      │   double   │
├─────────┼─────────────────────────────┼────────┼───────────┼───────────┼──────────────────┼────────────┤
│ UGA     │ POINT (32.5333 0.5833)      │    1.0 │    0.5833 │   32.5333 │ Bombo            │    75000.0 │
│ UGA     │ POINT (30.275 0.671)        │    2.0 │     0.671 │    30.275 │ Fort Portal      │    42670.0 │
│ ITA     │ POINT (15.799 40.642)       │    3.0 │    40.642 │    15.799 │ Potenza          │    69060.0 │
│ ITA     │ POINT (14.656 41.563)       │    4.0 │    41.563 │    14.656 │ Campobasso       │    50762.0 │
│ ITA     │ POINT (7.315 45.737)        │    5.0 │    45.737 │     7.315 │ Aosta            │    34062.0 │
│ ALD     │ POINT (19.949 60.097)    

## GeoJSON Files

In [22]:
con.sql('SELECT * FROM ST_Drivers()')

┌────────────────┬─────────────────────────────────────┬────────────┬──────────┬──────────┬────────────────────────────────────────────────┐
│   short_name   │              long_name              │ can_create │ can_copy │ can_open │                    help_url                    │
│    varchar     │               varchar               │  boolean   │ boolean  │ boolean  │                    varchar                     │
├────────────────┼─────────────────────────────────────┼────────────┼──────────┼──────────┼────────────────────────────────────────────────┤
│ ESRI Shapefile │ ESRI Shapefile                      │ true       │ false    │ true     │ https://gdal.org/drivers/vector/shapefile.html │
│ MapInfo File   │ MapInfo File                        │ true       │ false    │ true     │ https://gdal.org/drivers/vector/mitab.html     │
│ UK .NTF        │ UK .NTF                             │ false      │ false    │ true     │ https://gdal.org/drivers/vector/ntf.html       │
│ LVBAG      

In [23]:
con.sql("SELECT * FROM ST_Read('cities.geojson')")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┬─────────────────────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │            geom             │
│ int32 │     varchar      │ varchar │  double   │  double   │   int32    │          geometry           │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┼─────────────────────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │ POINT (32.5333 0.5833)      │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │ POINT (30.275 0.671)        │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │ POINT (15.799 40.642)       │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │ POINT (14.656 41.563)       │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │ POINT (7.315 45.737)        │
│     6 │ Mariehamn        │ ALD     │    60.0

In [24]:
con.sql("FROM ST_Read('cities.geojson')")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┬─────────────────────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │            geom             │
│ int32 │     varchar      │ varchar │  double   │  double   │   int32    │          geometry           │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┼─────────────────────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │ POINT (32.5333 0.5833)      │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │ POINT (30.275 0.671)        │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │ POINT (15.799 40.642)       │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │ POINT (14.656 41.563)       │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │ POINT (7.315 45.737)        │
│     6 │ Mariehamn        │ ALD     │    60.0

In [25]:
con.sql("CREATE TABLE cities AS SELECT * FROM ST_Read('cities.geojson')")

In [26]:
con.table('cities')

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┬─────────────────────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │            geom             │
│ int32 │     varchar      │ varchar │  double   │  double   │   int32    │          geometry           │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┼─────────────────────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │ POINT (32.5333 0.5833)      │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │ POINT (30.275 0.671)        │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │ POINT (15.799 40.642)       │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │ POINT (14.656 41.563)       │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │ POINT (7.315 45.737)        │
│     6 │ Mariehamn        │ ALD     │    60.0

In [27]:
con.sql("SELECT * FROM cities")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┬─────────────────────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │            geom             │
│ int32 │     varchar      │ varchar │  double   │  double   │   int32    │          geometry           │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┼─────────────────────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │ POINT (32.5333 0.5833)      │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │ POINT (30.275 0.671)        │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │ POINT (15.799 40.642)       │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │ POINT (14.656 41.563)       │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │ POINT (7.315 45.737)        │
│     6 │ Mariehamn        │ ALD     │    60.0

## Shapefiles

In [28]:
con.sql("SELECT * FROM ST_Read('cities.shp')")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┬─────────────────────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │            geom             │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │          geometry           │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┼─────────────────────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │ POINT (32.5333 0.5833)      │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │ POINT (30.275 0.671)        │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │ POINT (15.799 40.642)       │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │ POINT (14.656 41.563)       │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │ POINT (7.315 45.737)        │
│     6 │ Mariehamn        │ ALD     │    60.0

In [29]:
con.sql("FROM ST_Read('cities.shp')")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┬─────────────────────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │            geom             │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │          geometry           │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┼─────────────────────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │ POINT (32.5333 0.5833)      │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │ POINT (30.275 0.671)        │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │ POINT (15.799 40.642)       │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │ POINT (14.656 41.563)       │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │ POINT (7.315 45.737)        │
│     6 │ Mariehamn        │ ALD     │    60.0

In [30]:
con.sql(
    """
        CREATE TABLE IF NOT EXISTS cities2 AS
        SELECT * FROM ST_Read('cities.shp')
        """
)

In [31]:
con.table('cities2')

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┬─────────────────────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │            geom             │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │          geometry           │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┼─────────────────────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │ POINT (32.5333 0.5833)      │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │ POINT (30.275 0.671)        │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │ POINT (15.799 40.642)       │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │ POINT (14.656 41.563)       │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │ POINT (7.315 45.737)        │
│     6 │ Mariehamn        │ ALD     │    60.0

In [32]:
con.sql('SELECT * FROM cities2')

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┬─────────────────────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │            geom             │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │          geometry           │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┼─────────────────────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │ POINT (32.5333 0.5833)      │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │ POINT (30.275 0.671)        │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │ POINT (15.799 40.642)       │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │ POINT (14.656 41.563)       │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │ POINT (7.315 45.737)        │
│     6 │ Mariehamn        │ ALD     │    60.0

## GeoParquet Files

In [33]:
con.sql("SELECT * FROM 'cities.parquet'")

┌─────────┬─────────────────────────────┬────────┬───────────┬───────────┬──────────────────┬────────────┐
│ country │          geometry           │   id   │ latitude  │ longitude │       name       │ population │
│ varchar │          geometry           │ double │  double   │  double   │     varchar      │   double   │
├─────────┼─────────────────────────────┼────────┼───────────┼───────────┼──────────────────┼────────────┤
│ UGA     │ POINT (32.5333 0.5833)      │    1.0 │    0.5833 │   32.5333 │ Bombo            │    75000.0 │
│ UGA     │ POINT (30.275 0.671)        │    2.0 │     0.671 │    30.275 │ Fort Portal      │    42670.0 │
│ ITA     │ POINT (15.799 40.642)       │    3.0 │    40.642 │    15.799 │ Potenza          │    69060.0 │
│ ITA     │ POINT (14.656 41.563)       │    4.0 │    41.563 │    14.656 │ Campobasso       │    50762.0 │
│ ITA     │ POINT (7.315 45.737)        │    5.0 │    45.737 │     7.315 │ Aosta            │    34062.0 │
│ ALD     │ POINT (19.949 60.097)    

In [34]:
con.sql(
    """
CREATE TABLE IF NOT EXISTS cities3 AS
SELECT * EXCLUDE geometry, ST_GeomFromWKB(geometry)
AS geometry FROM 'cities.parquet'
"""
)

BinderException: Binder Error: No function matches the given name and argument types 'ST_GeomFromWKB(GEOMETRY)'. You might need to add explicit type casts.
	Candidate functions:
	ST_GeomFromWKB(WKB_BLOB) -> GEOMETRY
	ST_GeomFromWKB(BLOB) -> GEOMETRY

LINE 3: SELECT * EXCLUDE geometry, ST_GeomFromWKB(geometry) 
                                   ^

In [35]:
con.table('cities3')

CatalogException: Catalog Error: Table with name cities3 does not exist!
Did you mean "cities"?
LINE 1: from cities3
             ^

In [36]:
con.sql(
    """
CREATE TABLE IF NOT EXISTS country AS
SELECT * EXCLUDE geometry, ST_GeomFromWKB(geometry) FROM
        's3://us-west-2.opendata.source.coop/google-research-open-buildings/v2/geoparquet-admin1/country=SSD/*.parquet'
"""
)

BinderException: Binder Error: No function matches the given name and argument types 'ST_GeomFromWKB(GEOMETRY)'. You might need to add explicit type casts.
	Candidate functions:
	ST_GeomFromWKB(WKB_BLOB) -> GEOMETRY
	ST_GeomFromWKB(BLOB) -> GEOMETRY

LINE 3: SELECT * EXCLUDE geometry, ST_GeomFromWKB(geometry) FROM
                                   ^

In [37]:
con.table('country')

CatalogException: Catalog Error: Table with name country does not exist!
Did you mean "pg_constraint"?
LINE 1: from country
             ^

In [38]:
con.sql('SELECT COUNT(*) FROM country')

CatalogException: Catalog Error: Table with name country does not exist!
Did you mean "pg_constraint"?

## References

- [DuckDB Data Ingestion](https://duckdb.org/docs/api/python/data_ingestion)